# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [122]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

#### Creating list of filepaths to process original event csv data files

In [123]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [124]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [125]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [126]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [127]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [128]:
# TO-DO: Create a Keyspace 
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Set Keyspace

In [129]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_app_histoy 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### Connect to our KeySpace. 

In [130]:
try:
    session.set_keyspace('music_app_histoy')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

# Create queries to ask the following three questions of the data

## 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4



### Create and insert into table music_library_session

In [131]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS music_library_session "
query = query + "(artist text, song_title text, length double, sessionId int,  itemInSession int, PRIMARY KEY (sessionId,  itemInSession))"
try:
    session.execute(query)
    print("music_library_session created")
except Exception as e:
    print(e)

music_library_session created


In [132]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_session (artist, song_title, length, sessionId,  itemInSession)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], Decimal(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [133]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query  = """ SELECT sessionId, itemInSession, artist, song_title, length FROM music_library_session WHERE sessionId = 338  and itemInSession = 4""" 

try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
# collecting all rows in list as a tuple
allRows = [] 
for row in rows:
    allRows.append((row.artist, row.song_title,row.length))
# Create dataFrame for the result query
if(len(allRows) > 0):
    df_music_library_session = pd.DataFrame(allRows,columns=['artist','song_title','length'])   
df_music_library_session.head()
    

,artist,song_title,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073



### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182



### Create and insert into music_library_session_2

In [134]:
## TO-DO: Query 2: Give me only the following: 
## name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS music_library_session_2 "
query = query + "( userID int, sessionId int, itemInSession int, artist text, song_title text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession) )"
try:
    session.execute(query)
    print("music_library_session_2 created")
except Exception as e:
    print(e)
                    

music_library_session_2 created


In [135]:
df.dtypes

artist            object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
sessionId          int64
song              object
userId             int64
dtype: object

In [136]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_session_2 (userID, sessionId, itemInSession, artist, song_title, firstName, lastName)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[-1]), int(line[8]), int(line[3]), line[0], line[-2], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into music_library_session_2

In [137]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query  = """ SELECT artist, song_title, firstName, lastName FROM music_library_session_2 WHERE userID = 10 AND sessionId = 182 """ 

try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
print(row.artist)
# collecting all rows in list as a tuple
allRows = [] 
for row in rows:
    allRows.append((row.artist, row.song_title, row.firstname, row.lastname))
# # Create dataFrame for the result query
df_music_library_session_2 = pd.DataFrame(allRows,columns=['artist','song_title','firstName', 'lastName'])   
df_music_library_session_2.head()

Faithless


,artist,song_title,firstName,lastName
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Create music_library_session_3

In [138]:
df.dtypes

artist            object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
sessionId          int64
song              object
userId             int64
dtype: object

In [139]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS music_library_session_3 "
query = query + "( song_title text,  firstName text, lastName text, userID int, PRIMARY KEY (song_title, userID) )"
try:
    session.execute(query)
    print("music_library_session_3 created")
except Exception as e:
    print(e)

music_library_session_3 created


### ...insert

In [140]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_session_3 (song_title, firstName, lastName, userID)"
        query = query + " VALUES (%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[-2], line[4], line[1], int(line[-1])))

#### Do a SELECT to verify that the data have been inserted into music_library_session_3

In [119]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query  = """ SELECT firstName, lastName FROM music_library_session_3 WHERE song_title = 'All Hands Against His Own' ALLOW FILTERING """ 

try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
# print(row.artist)
# collecting all rows in list as a tuple
allRows = [] 
for row in rows:
    allRows.append(( row.firstname, row.lastname))
# # Create dataFrame for the result query
df_music_library_session_3 = pd.DataFrame(allRows, columns=['firstName', 'lastName'])   
df_music_library_session_3.head()

,firstName,lastName
0,Levine,Tegan
1,Johnson,Sara
2,Lynch,Jacqueline


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [141]:
session.execute("""DROP TABLE music_library_session_3""")

session.execute("""DROP TABLE music_library_session_2""")

session.execute("""DROP TABLE music_library_session""")

### Close the session and cluster connection¶

In [142]:
session.shutdown()
cluster.shutdown()